In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
177,AFG,Asia,Afghanistan,2020-07-05,32672.0,348.0,826.0,7.0,839.286,8.940,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83
296,ALB,Europe,Albania,2020-07-05,2819.0,67.0,74.0,2.0,979.568,23.282,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
479,DZA,Africa,Algeria,2020-07-05,15500.0,430.0,946.0,9.0,353.469,9.806,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-07-04,France,167698.0,0.0,29910.0,0.0,0.0,0.0
0,2020-07-04,China,84888.0,0.0,4651.0,0.0,0.0,0.0
0,2020-07-04,Italy,241517.0,0.0,34886.0,0.0,0.0,0.0
0,2020-07-04,Spain,251036.0,0.0,28406.0,0.0,0.0,0.0
0,2020-07-04,United States,2840966.0,0.0,130269.0,0.0,0.0,0.0
0,2020-07-04,World,11254739.0,0.0,532059.0,0.0,0.0,0.0
0,2020-07-04,United Kingdom,258495.0,0.0,44273.0,0.0,0.0,0.0
0,2020-07-04,Germany,196608.0,0.0,9028.0,0.0,0.0,0.0
0,2020-07-04,Iran,238610.0,0.0,11397.0,0.0,0.0,0.0
0,2020-07-04,Turkey,204570.0,0.0,5202.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-07-05,France,167698.0,166960.0,29910.0,29893.0,738.0,17.0
0,2020-07-05,China,84888.0,84857.0,4651.0,4641.0,31.0,10.0
0,2020-07-05,Italy,241517.0,241419.0,34886.0,34854.0,98.0,32.0
0,2020-07-04,Spain,251036.0,250545.0,28406.0,28385.0,491.0,21.0
0,2020-07-05,United States,2840966.0,2839542.0,130269.0,129676.0,1424.0,593.0
0,2020-07-05,World,11254739.0,11241655.0,532059.0,530668.0,13084.0,1391.0
0,2020-07-05,United Kingdom,258495.0,284900.0,44273.0,44198.0,-26405.0,75.0
0,2020-07-05,Germany,196608.0,196335.0,9028.0,9012.0,273.0,16.0
0,2020-07-05,Iran,238610.0,237878.0,11397.0,11408.0,732.0,-11.0
0,2020-07-05,Turkey,204570.0,204610.0,5202.0,5206.0,-40.0,-4.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")